# Observability of Primordial Asteroids Application
## (Primo Pole Project)
In the frame of <a href="https://mp3c.oca.eu/catalogue/projects.htm" target="_blank">Primo Pole Project </a> leaded by Dr. Marco Delbo from Observatoire de la Côte d'Azur (OCA), asteroids that belong to the Primordial family need to be observed, thus theri spin pole to be estimated. This family was discovered by Delbo et al. in 2017 and is published as "*Delbo, M., Walsh, K., Bolin, B., Avdellidou, C., & Morbidelli, A. (2017). Identification of a primordial asteroid family constrains the original planetesimal population. Science, 357(6355).*". This family could be as old as the Solar System, i.e. ~4.5 Gyears old.
The working hypothesis is that according to theories of asteroid orbital evolution, under the influence of the Yarkovsky effect, members of the inward side of the Primordial family should have a statistical predominance of retrograde objects.

**This code finds the observable asteroids, members of the Primordial family, based on the location, the observation period and instrument limitations.** In this way, a short list of candidate targets will be exported automatically providing us the capability to do a smart sampling.   

In [ ]:
#Useful Packages and Functions
import numpy as np
import pandas as pd
import astropy
from astroquery.jplhorizons import Horizons
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import datetime as dt #Package for the dates
from scipy.stats import norm #Package for statistics
#import seaborn as sns #Package Styling for Statistics
from bisect import bisect_left

#To export plots as images
import io
import urllib, base64
import time

#Packages for interaction
import ipywidgets as ipw
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import HTML, display, Javascript, clear_output
import nbinteract as nbi


progress=ipw.IntProgress(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Searching:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)


def take_closest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = pos - 1
    after = pos
    if myList[after] - myNumber < myNumber - myList[before]:
       return after
    else:
       return before


In [ ]:
################################################ MAIN CODE ########################################

#TITLE
Title=ipw.HTML('<p style="text-align: center;"><span style="font-size: 40px; font-family: Helvetica;"><span style="color: rgb(85, 57, 130);"><strong>O</strong>bservability of <strong>P</strong>rimordial <strong>A</strong>steroids</span> <strong>App</strong>lication</span></p>')
#Title.layout.height = '80px'
#Title.layout.align_content = 'center'

#COPYRIGHTS
Copyright=ipw.HTML('<footer style="text-align: right;">&copy; 2021 Dimitrios Athanasopoulos </footer>')
Copyright.layout.height = '15px'
#CONTACT
Contact=ipw.HTML('<footer style="text-align: right;">Contact: dimathanaso@phys.uoa.gr </footer>')
Contact.layout.height = '30px'

################################################ A PART ####################################################

#TAB_1
Location_text=ipw.HTML('<p style="text-align: left;"><strong><span style="font-size: 18px; font-family: Helvetica;">Insert your Location and Timezone</span></strong></p>')
LON_ft=ipw.FloatText(description="Longitude (deg):", value=23.783368, style={'description_width': '100px'})
LAT_ft=ipw.FloatText(description="Latitude (deg):", value=37.968561, style={'description_width': '100px'})
ALT_ft=ipw.FloatText(description="Altidute (km):", value=0.250, style={'description_width': '100px'})
TimeZone_dd=ipw.Dropdown(description="TimeZone:", options=['Etc/GMT+12', 'Etc/GMT+11', 'Etc/GMT+10', 'Etc/GMT+9', 'Etc/GMT+8', 'Etc/GMT+7', 'Etc/GMT+6',
                               'Etc/GMT+5', 'Etc/GMT+4', 'Etc/GMT+3', 'Etc/GMT+2', 'Etc/GMT+1', 'UTC',
                               'Etc/GMT-1', 'Etc/GMT-2', 'Etc/GMT-3', 'Etc/GMT-4', 'Etc/GMT-5', 'Etc/GMT-6', 'Etc/GMT-7', 
                               'Etc/GMT-8', 'Etc/GMT-9', 'Etc/GMT-10', 'Etc/GMT-11', 'Etc/GMT-12', 'Etc/GMT-13', 'Etc/GMT-14'],
                      value='UTC', style={'description_width': '100px'})
OutputMessage1a=ipw.HTML(value="")
OutputMessage1b=ipw.HTML(value="")

def location_input(b=None):
    global LON, LAT, ALT, TimeZone, OutputMessage1a, InsertLocation
    LON=LON_ft.value; LAT=LAT_ft.value; ALT=ALT_ft.value; TimeZone=TimeZone_dd.value
    clear_output(wait=True)
    OutputMessage1a.value="Your Location is: Longitude: " + str(LON) + " deg --- Latitude: " + str(LAT) + " deg --- Altitude: " + str(ALT) + " km"
    OutputMessage1b.value="Your TimeZone is: " + TimeZone
    display(DashBoard1)

Insert_btn1=ipw.Button(description="Insert", button_style='Success')

Insert_btn1.on_click(location_input)

InsertLocation=ipw.VBox([Location_text, LON_ft, LAT_ft, ALT_ft, TimeZone_dd, Insert_btn1, OutputMessage1a, OutputMessage1b])

#TAB_2
ObservationPeriod_text=ipw.HTML('<p style="text-align: left;"><strong><span style="font-size: 18px; font-family: Helvetica;">Insert your Observation period </span></strong></p>')
START_t=ipw.Text(description="Start date (YYYY-MM-DD):", style={'description_width': '180px'})
STOP_t=ipw.Text(description="Stop date (YYYY-MM-DD):", style={'description_width': '180px'})
OutputMessage2=ipw.HTML(value="")

def observationPeriod_input(b=None):
    global START, STOP
    START=START_t.value; STOP=STOP_t.value
    clear_output(wait=True)
    OutputMessage2.value="Your Observation Period is from " + START + " at 12:00 (UTC) to " + STOP + " at 12:00 (UTC)."
    display(DashBoard1)


Insert_btn2=ipw.Button(description="Insert", button_style='Success')

Insert_btn2.on_click(observationPeriod_input)

InsertObservationPeriod=ipw.VBox([ObservationPeriod_text, START_t, STOP_t, Insert_btn2, OutputMessage2])

#TAB_3
Preferences_text=ipw.HTML('<p style="text-align: left;"><strong><span style="font-size: 18px; font-family: Helvetica;">Insert your instrument limitations and preferences</span></strong></p>')
APmag_min_ft=ipw.FloatText(description="The target max. ap. magnitude (mag):", value=17, style={'description_width': '230px'})
MinObsTime_ft=ipw.FloatText(description="The minimum observation time (hours):", value=4, style={'description_width': '230px'})
OutputMessage3=ipw.HTML("")

def preferences_input(b=None):
    global APmag_min, MinObsTime
    
    APmag_min=APmag_min_ft.value; MinObsTime=MinObsTime_ft.value
    clear_output(wait=True)
    OutputMessage3.value="You want to observe targets with ≤ " + str(APmag_min) + " mag for at least " + str(MinObsTime) + " hours."
    display(DashBoard1)
    #display(InsertPreferences)
    #print("You want to observe targets with ≤ " + str(APmag_min) + " mag for at least " + str(MinObsTime) + " hours.")


Insert_btn3=ipw.Button(description="Insert", button_style='Success')
Insert_btn3.on_click(preferences_input)

InsertPreferences=ipw.VBox([Preferences_text, APmag_min_ft, MinObsTime_ft, Insert_btn3, OutputMessage3])


#ALL_TABS
InsertTab=ipw.Tab([InsertLocation, InsertObservationPeriod, InsertPreferences])
InsertTab.set_title(0, 'Your Location')
InsertTab.set_title(1, 'Observation Period')
InsertTab.set_title(2, 'Preferences')

IDtargetList=[]



#FIND_OBSERVABLE_ASTEROIDS

def find_asteroids(b=None):
    global Family, KnownFamily, TargetList, IDtargetList, Observatory, ObsDays, OptionTargets, FIG1, OUT1
    ct1=0 #This is for the final button
    
    if (OutputMessage1a.value==""):
        OutputMessage4.value='<p><strong><span style="color: red;">Please insert your Location and Timezone!</span></strong></p>'
    elif (OutputMessage2.value=="") or (START_t.value=="") or (STOP_t.value==""):
        OutputMessage4.value='<p><strong><span style="color: red;">Please insert your Observation Period!</span></strong></p>'
    elif (OutputMessage3.value==""):
        OutputMessage4.value='<p><strong><span style="color: red;">Please insert your Preferences!</span></strong></p>'
    else:
        Observatory={'lon': LON, 'lat': LAT, 'elevation': ALT}
        step=1 #Time Step
        units="h"#Time units, type h: for hours, m: for minutes, d: for days
        ObsDays={'start':START+' 12:00', 'stop':STOP+' 12:00','step':(str(step)+units)}
        
        if units=='h':
            step=step
        elif units=='m':
            step=step/60
        elif units=='d':
            step=step*24
        else:
            print('WARNING!: Please insert again a time-step with units in minutes("m") or hours ("h") or days ("d")')
        
        #READ FILE
        Family = pd.read_excel('PrimordialFamily.xlsx', sheet_name='All_Members')
        
        OutputMessage4.value='<p><strong><span style="color: green;">Initialisation completed!</span></strong></p>'
        
        #DASHBOARD_1 (with progress bar)
        #clear_output(wait=True)
        progress.layout.visibility = 'visible'
        #DashBoard2=ipw.VBox([Title, Copyright, Contact, InsertTab, FindAsteroids, progress])
        #display(DashBoard1)
        
        #Search for Observable Asteroids
        TargetList=[]
        IDtargetList=[]
        
        it=0
        progress.max=(len(Family['Number'])-1)
        for IDtarget in Family['Number']:
            progress.value=it
            progress.value=it
            
            #Find epherides from Horizon - JPL Small Bodies Solar System database
            obj = Horizons(id=int(IDtarget), location=Observatory, epochs=ObsDays)
            eph = obj.ephemerides()
            eph=eph.to_pandas()
            
            #This is the full name of the target
            TargetName=eph['targetname'][0]
            
            #Datetime formatting
            Date=eph['datetime_str']
            DateT=pd.to_datetime(Date)
            if (TimeZone=='UTC') | (TimeZone=='Europe/London'):
                LocalDate=DateT
            else:
                LocalDate = DateT.dt.tz_localize('UTC').dt.tz_convert(TimeZone)
            
            #Determine the nightime
            Sun=eph['solar_presence']
            NightTime=[]
            n=0
            for x in range(len(Sun)):
                if (Sun[x]==""):
                    NightTime.append(LocalDate[x])
                    n=+1
            
            #The target at nightime
            Target=eph[['datetime_str', 'solar_presence','V', 'airmass','RA','DEC','alpha_true','PABLon','PABLat']].copy(deep=True) #Suppress the SettingWithCopyWarning
            Target['datetime_str']=pd.to_datetime(Target['datetime_str'])
            pd.options.mode.chained_assignment = None  # default='warn'
            Target.rename(columns={'datetime_str':'datetime'}, inplace=True)
            Target=Target[(Target['airmass']<=3) & (Target['solar_presence']=="")]
            Target.reset_index(drop=True, inplace=True)
            
            #To release memory
            del eph
            
            #Duration of observable target
            DurationT=[]
            if len(Target)>1 and abs(Target["datetime"][0]-Target["datetime"][len(Target["datetime"])-1])<dt.timedelta(hours=24):
                check=1
                c1=0
                c2=len(Target["datetime"])-1
                D0=(pd.Timestamp.date(Target["datetime"][c2]))
                dd=Target["datetime"][c2]-Target["datetime"][c1]
                D1=(dd.total_seconds()/3600)
                D2=(Target["V"][int((c2+c1)/2)])
                if (TimeZone=='UTC') | (TimeZone=='Europe/London'):
                    D3=(Target["datetime"][c1])
                    D4=(Target["datetime"][c2])
                else:
                    D3=(Target["datetime"][c1].tz_localize(TimeZone))
                    D4=(Target["datetime"][c2].tz_localize(TimeZone))
                D5=([Target["RA"][c1],Target["DEC"][c1]])
                D6=([Target["RA"][c2],Target["DEC"][c2]])
                D7=(Target["alpha_true"][int((c2+c1)/2)])
                D8=(Target["PABLon"][int((c2+c1)/2)])
                D9=(Target["PABLat"][int((c2+c1)/2)])
                DurationT.append([D0,D1,D2,D3,D4,D5,D6,D7,D8,D9])
            else:
                c1=0
                c2=0
                check=0
                for i in range(len(Target["datetime"])-1):
                    if  abs(Target["datetime"][i]-Target["datetime"][i+1])>dt.timedelta(hours=step):
                        check=1
                        c2=i
                        D0=(pd.Timestamp.date(Target["datetime"][c2]))
                        dd=Target["datetime"][c2]-Target["datetime"][c1]
                        D1=(dd.total_seconds()/3600)
                        D2=(Target["V"][int((c2+c1)/2)])
                        if (TimeZone=='UTC') | (TimeZone=='Europe/London'):
                            D3=(Target["datetime"][c1])
                            D4=(Target["datetime"][c2])
                        else:
                            D3=(Target["datetime"][c1].tz_localize(TimeZone))
                            D4=(Target["datetime"][c2].tz_localize(TimeZone))
                        D5=([Target["RA"][c1],Target["DEC"][c1]])
                        D6=([Target["RA"][c2],Target["DEC"][c2]])
                        D7=(Target["alpha_true"][int((c2+c1)/2)])
                        D8=(Target["PABLon"][int((c2+c1)/2)])
                        D9=(Target["PABLat"][int((c2+c1)/2)])
                        DurationT.append([D0,D1,D2,D3,D4,D5,D6,D7,D8,D9])
                        c1=c2+1
                    
            if (check==1):
                colD=['Mid-Date', 'Duration (h)', 'V (mag)', 'Observation Start (UTC + Timezone)', 'Observation End (UTC + Timezone)', 'Start Point [RA,DEC]', 'End Point [RA,DEC]', 'Solar Phase (\u00b0)','PABLon (\u00b0)','PABLat (\u00b0)']
                DurationT=pd.DataFrame(DurationT, columns=colD)
                if(max(DurationT["Duration (h)"])>=MinObsTime) & (min(DurationT["V (mag)"])<=APmag_min):
                    TargetList.append([TargetName, max(DurationT["Duration (h)"]), min(DurationT["V (mag)"]), Family["P (h)"][it], Family["lambda 1 (deg)"][it], Family["beta 1 (deg)"][it], Family["lambda 2 (deg)"][it], Family["beta 2 (deg)"][it]])
                    IDtargetList.append(IDtarget)
        
            it=it+1
                    
        
        colTL=['Target', 'Maximum Duration (h)', 'Minimum V (mag)', 'P (h)', '$\lambda_1$ (deg)', '$\\beta_1$ (deg)', '$\lambda_2$ (deg)', '$\\beta_2$ (deg)' ]
        TargetList=pd.DataFrame(TargetList, columns=colTL)
        OptionTargets=list(zip(TargetList['Target'].tolist(),IDtargetList))
        
        #PRIORITY_RATE
        ApAll=Family["a_p (au)"]
        KnownFamily=Family[Family["lambda 1 (deg)"]!='-'] #Creating a new dataframe with the family members with known spin
        ApK=KnownFamily["a_p (au)"]
        
        #Density distribution of all family members
        Ap = np.linspace(min(ApAll), max(ApAll), 1000)
        densityA = sum(norm.pdf(Ap, xi, 0.02) for xi in ApAll)
        
        #Density distribution of family members with known spin 
        densityK = sum(norm.pdf(Ap,xi, 0.02) for xi in ApK)
        
        #Definition of priority rate for an homogenous observing sample
        PR=np.floor(100*(max(densityK/densityA)-densityK/densityA))
        TL=[]
        for ta in range(len(IDtargetList)):
            cin=Family[Family["Number"]==IDtargetList[ta]].index
            xa=take_closest(Ap, Family["a_p (au)"][cin].tolist())
            if Family["lambda 1 (deg)"][cin].tolist()==['-']:
                TL.append(PR[xa])
            else:
                TL.append(0)
        
        TargetList["Priority Rate (%)"]=TL
        
        TargetList=TargetList.sort_values(["Priority Rate (%)", "Maximum Duration (h)", "Minimum V (mag)"], ascending = (False, False, True))
        #print('The targets are display here with a priority row:')
        #TargetListBest.head(len(TargetList))
        
        # Turn interactive plotting off
        plt.ioff()
        
        CenterOfFamily=2.366
        fig1 = plt.figure(figsize=(15,3))
        
        #Make room for x label
        #plt.tight_layout()
        plt.gcf().subplots_adjust(bottom=0.25)
        
        plt.axvline(0, color='grey', linestyle='--')
        plt.text(-0.01,2*max(100*densityK/densityA)/3,"$a_c="+str(CenterOfFamily)+"\ au$", bbox=dict(facecolor='white', alpha=0.95))
        plt.fill_between(Ap-CenterOfFamily, 100*(max(densityK/densityA)-densityK/densityA), alpha=0.5)
        plt.plot(ApAll-CenterOfFamily, np.full_like(ApAll, -1), '+k', markersize=10 , markeredgewidth=1, label='All members')
        plt.plot(ApK-CenterOfFamily, np.full_like(ApK, -1), '+r', markersize=10 , markeredgewidth=3, label='Members with known spin')
        plt.title("Priority Rate for the Primordial Family members for an homogenous observing sample")
        plt.ylabel("Priority Rate (%)")
        plt.xlabel("Position from the centre of the family (au)")
        plt.legend()
        plt.axis([min(ApAll-CenterOfFamily)-0.01, max(ApAll-CenterOfFamily)+0.01, -2, max(100*densityK/densityA)])
        plt.close(fig1)
        
        #Export plot as HTML image
        buf1 = io.BytesIO()
        fig1.savefig(buf1, format='png')
        buf1.seek(0)
        string1 = base64.b64encode(buf1.read())
        
        uri1 = 'data:image/png;base64,' + urllib.parse.quote(string1)
        FIG1.value='<img src = "%s"/>' % uri1
        
        
        #DASHBOARD_1F (without progressbar, include taable and graph)
        clear_output(wait=True)
        progress.layout.visibility = 'hidden'
        #DashBoard1f=ipw.VBox([Title, Copyright, Contact, InsertTab, FindAsteroids])
        display(DashBoard1)
        display(HTML(TargetList.to_html(index=False)))
        #display(HTML('<p><span style="font-size: 10px; font-family: Helvetica;">The ephimeris data were computed using the <a href="https://ssd.jpl.nasa.gov/?horizons" rel="noopener noreferrer" target="_blank">JPL&apos;s HORIZON system</a>, physical parametrers have been retrieved from the <a href="https://mp3c.oca.eu/catalogue/index.htm" rel="noopener noreferrer" target="_blank">Minor Planet Physical Properties Catalogue</a> (Delbo, Walsh, Bolin, Avdellidou, Morbidelli. 2017. &quot;Identification of a primordial asteroid family constrains the original planetesimal population&quot;. Science. 357, 1026 - 1029) and the spin pole data from the <a href="https://astro.troja.mff.cuni.cz/projects/damit/" rel="noopener noreferrer" target="_blank">DAMIT</a> (Ďurech, J., Sidorin, V., &amp; Kaasalainen, M. (2010). DAMIT: a database of asteroid models. Astronomy &amp; Astrophysics, 513, A46).</span></p>'))
        display(TableFootNote)
        
        display(FIG1)
        

        #DASHBOARD_2
        IDtarget_dd.options=OptionTargets #IDtargetList
        DashBoard2.layout.display = ''
        display(DashBoard2)


Find_btn=ipw.Button(description="Find the Observable Asteroids", 
                    layout=ipw.Layout(justify_content='center', width='50%', height='50px'))
Find_btn.style.button_color = 'mediumpurple'
Find_btn.style.font_weight='bold'
OutputMessage4=ipw.HTML("")

TableFootNote=ipw.HTML('<p><span style="font-size: 10px; font-family: Helvetica;">The ephimeris data were computed using the <a href="https://ssd.jpl.nasa.gov/?horizons" rel="noopener noreferrer" target="_blank">JPL&apos;s HORIZON system</a>, physical parametrers have been retrieved from the <a href="https://mp3c.oca.eu/catalogue/index.htm" rel="noopener noreferrer" target="_blank">Minor Planet Physical Properties Catalogue</a> (Delbo, Walsh, Bolin, Avdellidou, Morbidelli. 2017. &quot;Identification of a primordial asteroid family constrains the original planetesimal population&quot;. Science. 357, 1026 - 1029) and the spin pole data from the <a href="https://astro.troja.mff.cuni.cz/projects/damit/" rel="noopener noreferrer" target="_blank">DAMIT</a> (Ďurech, J., Sidorin, V., &amp; Kaasalainen, M. (2010). DAMIT: a database of asteroid models. Astronomy &amp; Astrophysics, 513, A46).</span></p>')
        

FIG1=ipw.HTML("")

Find_btn.on_click(find_asteroids)

FindAsteroids=ipw.HBox([Find_btn, OutputMessage4])
FindAsteroids.layout.align_items = 'center'

#DASHBOARD_1
progress.layout.visibility = 'hidden'
DashBoard1=ipw.VBox([Title,  Copyright, Contact, InsertTab, FindAsteroids, progress])
display(DashBoard1)
#OUT1=ipw.Output()

################################################ B PART ####################################################
VOID=ipw.HTML("")
VOID.layout.height = '20px'

#Please type the ID of the target
TargetSelection_text=ipw.HTML('<p style="text-align: left;"><strong><span style="font-size: 18px; font-family: Helvetica;">Select your target</span></strong></p>')
IDtarget_dd=ipw.Dropdown(description="ID Target:", options=IDtargetList, style={'description_width': '100px'})
FIG2=ipw.HTML("")

def targetSelection_input(b=None):
    global IDtarget, ct1
    ct1=0 #This is for the final button
    
    IDtarget=IDtarget_dd.value
    cin=Family[Family["Number"]==IDtarget].index
    #clear_output(wait=True)
    #plt.close(fig)
    #display(TargetSelection)
    #Show where is located
    
    #plt.close()
    
    #clear_output(True)
    
    # Turn interactive plotting off
    plt.ioff()
    cm = plt.cm.get_cmap('jet')
    fig2,ax2 = plt.subplots(1,1, figsize=(10,5))

    #ax2.clear()
    plt.scatter(Family["a_p (au)"], Family["1/D (km^-1)"], color='k',s=10, label="Primordial Family members")
    sc1 = plt.scatter(KnownFamily["a_p (au)"], KnownFamily["1/D (km^-1)"], c=KnownFamily["beta 1 (deg)"], edgecolor='black', vmin=-90, vmax=90, s=50, cmap=cm, label="Members with known spin")
    plt.plot([0,2.366], [1.39594, 0], '-.k')
    plt.text(2.22,0.05,"Primordial Family \n (left border)", ha='center', va='center')
    plt.plot([0,2.42], [2.662, 0], '--k')
    plt.plot([2.42, 3], [0, 3*1.1-2.662], '--k')
    plt.text(2.42,0.125,"Polana Family \n borders", ha='center', va='center')
    plt.scatter(Family["a_p (au)"][cin], Family["1/D (km^-1)"][cin], marker='x', color='m', label="Selected Target")
    
    plt.xlabel("Proper semimajor axis (au)")
    plt.ylabel("$1/D \ (km^{-1})$")
    plt.xlim([2.12,2.5])
    plt.ylim([0, 0.28])
    #plt.yticks(np.arange(0, 0.200001, step=0.05))
    v = np.linspace(-90, 90, 10, endpoint=True)
    
    #if ct==0:
    cb=plt.colorbar(sc1, ticks=v)
    cb.set_label("$ \\beta \ (^o)$")
    
    plt.legend(loc='upper right', framealpha=0.95)
    plt.close(fig2)
    
    #Export plot as HTML image
    buf2 = io.BytesIO()
    fig2.savefig(buf2, format='png')
    buf2.seek(0)
    string2 = base64.b64encode(buf2.read())
        
    uri2 = 'data:image/png;base64,' + urllib.parse.quote(string2)
    #time.sleep(1)
    FIG2.value = '<img src = "%s"/>' % uri2  
    
    clear_output(wait=True)
    
    #Previous Output
    display(DashBoard1)
    display(HTML(TargetList.to_html(index=False)))
    display(TableFootNote)
    display(FIG1)
    display(DashBoard2)
    
    display(FIG2)
    
    #DASHBOARD_3
    #if ct==0:
    DashBoard3.layout.display = ''
    display(DashBoard3)
    
    #ct=ct+1


#IDtarget_dd=ipw.interact(description="ID Target:", options=IDtargetList, style={'description_width': '100px'})

Insert_btn4=ipw.Button(description="Insert", button_style='Success')
#out = ipw.Output()
#out

TargetSelection=ipw.VBox([TargetSelection_text, ipw.HBox([IDtarget_dd, Insert_btn4])])
#clear_output(wait=True)
#clear_output()
#display(TargetSelection)

#Click Times
ct=0;

Insert_btn4.on_click(targetSelection_input)

#DASHBOARD_2
DashBoard2=ipw.VBox([VOID, TargetSelection])
DashBoard2.layout.display = 'None'
display(DashBoard2)


################################################ C PART ####################################################

ObservationPlan_text=ipw.HTML('<p style="text-align: left;"><strong><span style="font-size: 18px; font-family: Helvetica;">The Observation Plan</span></strong></p>')


TimeStep=ipw.FloatText(description="Time step:", value=10)
TimeUnit=ipw.Dropdown(options=["m", "h", "d"], value="m")

TimeStepUnit=ipw.HBox([TimeStep, TimeUnit])

FIG3=ipw.HTML("")
FIG4=ipw.HTML("")

ct1=0

def create_obsplan(b=None):
    
    time.sleep(0.1) #Refresh
    OutputMessage5.value = '<p><strong><span style="color: blue;">Please wait...</span></strong></p>'
    
    global FIG3, FIG4, NightTime, LocalDate, airmass
    
    step=TimeStep.value
    units=TimeUnit.value
    
    ObsDays={'start':START+' 12:00', 'stop':STOP+' 12:00','step':(str(int(step))+units)}
    obj = Horizons(id=IDtarget, location=Observatory, epochs=ObsDays)
    eph = obj.ephemerides()
    TargetName=eph['targetname'][0]
    
    if units=='h':
        step=step
    elif units=='m':
        step=step/60
    elif units=='d':
        step=step*24
    else:
        print('WARNING!: Please insert again a time-step with units in minutes("m") or hours ("h") or days ("d")')
    
    eph=eph.to_pandas()
    airmass=eph['airmass']
    Date=eph['datetime_str']
    DateT=pd.to_datetime(Date)
    
    if (TimeZone=='UTC') | (TimeZone=='Europe/London'):
        LocalDate=DateT
    else:
        #LocalDate = DateT.dt.tz_localize('UTC').dt.tz_convert(TimeZone)
        LocalDate = DateT
    
    Sun=eph['solar_presence']
    NightTime=[]
    n=0
    for x in range(len(Sun)):
        if (Sun[x]==""):
            #print(LocalDate[x])
            NightTime.append(LocalDate[x])
            n=+1
    
    #The target at nightime
    Target=eph[['datetime_str', 'solar_presence','V', 'airmass','RA','DEC','alpha_true','PABLon','PABLat']].copy(deep=True) #Suppress the SettingWithCopyWarning
    Target['datetime_str']=pd.to_datetime(Target['datetime_str'])
    pd.options.mode.chained_assignment = None  # default='warn'
    Target.rename(columns={'datetime_str':'datetime'}, inplace=True)
    Target=Target[(Target['airmass']<=3) & (Target['solar_presence']=="")]
    Target.reset_index(drop=True, inplace=True)
    
    # Turn interactive plotting off
    plt.ioff()
    
    #if ct1==0:
    
    fig3, ax3 = plt.subplots(2,1, figsize=(20,15), sharex=True)
    #ax3[i].clear()
    ax3[0].plot(LocalDate,airmass)
    #ax3[i].fill_between(NightTime, [1]*len(NightTime), 0, facecolor='green', alpha=0.5)
    #plt.fill_between(NightTime,[1]*len(NightTime))  
    
    ax3[0].set_xlabel("Epoch (UTC)")
    ax3[0].set_ylabel("Airmass")
    ax3[0].set_title("Observing Window for " + TargetName + "\n" +"Location: (" + str(LAT) + "," + str(LON) + ")")
    ax3[0].set_xlim([LocalDate[0],LocalDate[len(LocalDate)-1]])
    ax3[0].set_ylim([1, 3])
    ax3[0].invert_yaxis()
    
    ax3[1].plot(LocalDate,eph["V"]) 
    
    ax3[1].set_xlabel("Epoch (UTC)")
    ax3[1].set_ylabel("Aparent Magnitude (mag)")
    ax3[1].set_title("Aparent Magnitude for " + TargetName + "\n" +"Location: (" + str(LAT) + "," + str(LON) + ")")
    ax3[1].set_xlim([LocalDate[0],LocalDate[len(LocalDate)-1]])
    ax3[1].invert_yaxis()
    
    for i, ax in enumerate(ax3.flatten()):
        for j in range(len(NightTime)-1):  #highlight from j to j+1
            if abs(NightTime[j]-NightTime[j+1])<=dt.timedelta(hours=step):
                pn3=ax.axvspan(NightTime[j], NightTime[j+1], facecolor='green', edgecolor='none', alpha=0.5)
        pn3.set_label('Nighttime')
        ax.legend(loc='upper right') 
        
        if (LocalDate[len(LocalDate)-1]-LocalDate[0])<=dt.timedelta(days=90):
            ml = MultipleLocator(1)
            ax3[i].xaxis.set_major_locator(ml)
            ax3[i].grid(which='major', axis='x', linestyle='-', linewidth=1.5,  color='#000000')
            ax3[i].xaxis.set_major_locator(mpl.dates.DayLocator())
            ax3[i].xaxis.set_major_formatter(mpl.dates.DateFormatter('%d %b %Y'))
            ax3[i].xaxis.set_tick_params(which='major', pad=40)
        
        if (LocalDate[len(LocalDate)-1]-LocalDate[0])>=dt.timedelta(days=20):
            plt.gcf().autofmt_xdate()
        
        if (LocalDate[len(LocalDate)-1]-LocalDate[0])<=dt.timedelta(days=1):
            ml = MultipleLocator(1/24)
            ax3[i].xaxis.set_minor_locator(ml)
            ax3[i].grid(which='minor', axis='x', linestyle='-', linewidth=0.5)
            ax3[i].xaxis.set_minor_formatter(mpl.dates.DateFormatter('%H:%M')) 
            plt.setp(ax3[i].xaxis.get_minorticklabels(), rotation=90)
        elif (LocalDate[len(LocalDate)-1]-LocalDate[0])<=dt.timedelta(days=5):
            ml = MultipleLocator(1/12)
            ax3[i].xaxis.set_minor_locator(ml)
            ax3[i].grid(which='minor', axis='x', linestyle='-', linewidth=0.5)
            ax3[i].xaxis.set_minor_formatter(mpl.dates.DateFormatter('%H:%M')) 
            plt.setp(ax3[i].xaxis.get_minorticklabels(), rotation=90)
        elif (LocalDate[len(LocalDate)-1]-LocalDate[0])<=dt.timedelta(days=10):
            ml = MultipleLocator(1/8)
            ax3[i].xaxis.set_minor_locator(ml)
            ax3[i].grid(which='minor', axis='x', linestyle='-', linewidth=0.5)
            ax3[i].xaxis.set_minor_formatter(mpl.dates.DateFormatter('%H:%M')) 
            plt.setp(ax3[i].xaxis.get_minorticklabels(), rotation=90)
        elif (LocalDate[len(LocalDate)-1]-LocalDate[0])<=dt.timedelta(days=15):
            ml = MultipleLocator(1/6)
            ax3[i].xaxis.set_minor_locator(ml)
            ax3[i].grid(which='minor', axis='x', linestyle='-', linewidth=0.5)
            ax3[i].xaxis.set_minor_formatter(mpl.dates.DateFormatter('%H:%M')) 
            plt.setp(ax3[i].xaxis.get_minorticklabels(), rotation=90)
        elif (LocalDate[len(LocalDate)-1]-LocalDate[0])<=dt.timedelta(days=20):
            ml = MultipleLocator(1/3)
            ax3[i].xaxis.set_minor_locator(ml)
            ax3[i].grid(which='minor', axis='x', linestyle='-', linewidth=0.5)
            ax3[i].xaxis.set_minor_formatter(mpl.dates.DateFormatter('%H:%M')) 
            plt.setp(ax3[i].xaxis.get_minorticklabels(), rotation=90)
        elif (LocalDate[len(LocalDate)-1]-LocalDate[0])<=dt.timedelta(days=30):
            ml = MultipleLocator(1/2)
            ax3[i].xaxis.set_minor_locator(ml)
            ax3[i].grid(which='minor', axis='x', linestyle='-', linewidth=0.5)
            ax3[i].xaxis.set_minor_formatter(mpl.dates.DateFormatter('%H:%M')) 
            plt.setp(ax3[i].xaxis.get_minorticklabels(), rotation=90)
    plt.draw()
    plt.close(fig3)
        
    #Export plot as HTML image
    buf3 = io.BytesIO()
    fig3.savefig(buf3, format='png')
    buf3.seek(0)
    string3 = base64.b64encode(buf3.read())
        
    uri3 = 'data:image/png;base64,' + urllib.parse.quote(string3)
    FIG3.value='<img src = "%s"/>' % uri3
    
    # Turn interactive plotting off
    #plt.ioff()
    
    #if ct1==0:
    #fig4, ax4 = plt.subplots(1,1, figsize=(20,4))
    #ax4.clear()    
    
    
    
    
    #CREATE THE OBSERVATION PLAN TABLE
    DurationT=[] 
    if abs(Target["datetime"][0]-Target["datetime"][len(Target["datetime"])-1])<dt.timedelta(hours=24):
        #check=1
        c1=0
        c2=len(Target["datetime"])-1
        if (pd.Timestamp.date(Target["datetime"][c1])==pd.Timestamp.date(Target["datetime"][c2])):
            D0=(pd.Timestamp.date(Target["datetime"][c2])+dt.timedelta(days=1))
        else:
            D0=(pd.Timestamp.date(Target["datetime"][c2]))
        dd=Target["datetime"][c2]-Target["datetime"][c1]
        D1=(dd.total_seconds()/3600)
        D2=(Target["V"][int((c2+c1)/2)])
        if (TimeZone=='UTC') | (TimeZone=='Europe/London'):
            D3=(Target["datetime"][c1])
            D4=(Target["datetime"][c2])
        else:
            D3=(Target["datetime"][c1].tz_localize(TimeZone))
            D4=(Target["datetime"][c2].tz_localize(TimeZone))
        D5=([Target["RA"][c1],Target["DEC"][c1]])
        D6=([Target["RA"][c2],Target["DEC"][c2]])
        D7=(Target["alpha_true"][int((c2+c1)/2)])
        D8=(Target["PABLon"][int((c2+c1)/2)])
        D9=(Target["PABLat"][int((c2+c1)/2)])
        DurationT.append([D0,D1,D2,D3,D4,D5,D6,D7,D8,D9])
    else:
        c1=0
        c2=0
        check=0
        for i in range(len(Target["datetime"])-1):
            if  abs(Target["datetime"][i]-Target["datetime"][i+1])>dt.timedelta(hours=step) or i==len(Target["datetime"])-2:
                #check=1
                c2=i
                if (pd.Timestamp.date(Target["datetime"][c1])==pd.Timestamp.date(Target["datetime"][c2])):
                    D0=(pd.Timestamp.date(Target["datetime"][c2])+dt.timedelta(days=1))
                else:
                    D0=(pd.Timestamp.date(Target["datetime"][c2]))
                dd=Target["datetime"][c2]-Target["datetime"][c1]
                D1=(dd.total_seconds()/3600)
                D2=(Target["V"][int((c2+c1)/2)])
                if (TimeZone=='UTC') | (TimeZone=='Europe/London'):
                    D3=(Target["datetime"][c1])
                    D4=(Target["datetime"][c2])
                else:
                    D3=(Target["datetime"][c1].tz_localize(TimeZone))
                    D4=(Target["datetime"][c2].tz_localize(TimeZone))
                D5=([Target["RA"][c1],Target["DEC"][c1]])
                D6=([Target["RA"][c2],Target["DEC"][c2]])
                D7=(Target["alpha_true"][int((c2+c1)/2)])
                D8=(Target["PABLon"][int((c2+c1)/2)])
                D9=(Target["PABLat"][int((c2+c1)/2)])
                DurationT.append([D0,D1,D2,D3,D4,D5,D6,D7,D8,D9])
                c1=c2+1
    
    col=['Mid-Date', 'Duration (h)', 'V (mag)', 'Observation Start (UTC + Timezone)', 'Observation End (UTC + Timezone)', 'Start Point [RA,DEC]', 'End Point [RA,DEC]', 'Solar Phase (\u00b0)','PABLon (\u00b0)','PABLat (\u00b0)']
    
    DurationT=pd.DataFrame(DurationT, columns=col)
    
    clear_output(wait=True)
    OutputMessage5.value=""
    
    #Previous Output
    display(DashBoard1)
    display(HTML(TargetList.to_html(index=False)))
    display(TableFootNote)
    display(FIG1)
    display(DashBoard2)
    display(FIG2)
    display(DashBoard3)

    
    #DASHBOARD_4
    #TimeStepUnit.layout.display = 'None' 
    #ObPlan_btn.layout.display = 'None'
    
    Message1= ipw.HTML('<p style="text-align: right;"> </p>' + "Location: (" + str(LAT) + "," + str(LON) + ") \t Timezone: " +TimeZone)
    Message2= ipw.HTML('The target '+TargetName+ ' is observable for these dates:')
    
    display(ipw.VBox([Message1, Message2]))
    #display(ipw.HTML(value=Message2))
    display(HTML(DurationT.to_html(index=False)))
    
    display(FIG3)
    #display(FIG4)
    
    #display(VOID)
    #AsteroidFinder_text=ipw.HTML('<p style="text-align: left;"><strong><span style="font-size: 18px; font-family: Helvetica;">Here you can find the path of the asteroid in the sky:</span></strong></p>')
    #%%html
    #<iframe src="https://asteroid.lowell.edu/astfinder/" frameborder="0" width="1800" height="1000"></iframe>
    
    #display(VOID)
    #AsteroidFinder_text=ipw.HTML('<p style="text-align: left;"><strong><span style="font-size: 18px; font-family: Helvetica;">Here you can check archival lightcurves:</span></strong></p>')
    #%%html
    #<iframe src="http://alcdef.org/" frameborder="0" width="1800" height="1000"></iframe>
    
    
    #ct1=ct1+1



ObPlan_btn=ipw.Button(description="Create the Observation Plan", 
                    layout=ipw.Layout(justify_content='center', width='50%', height='50px'))
ObPlan_btn.style.button_color = 'mediumslateblue'
ObPlan_btn.style.font_weight='bold'

ObPlan_btn.on_click(create_obsplan)

OutputMessage5=ipw.HTML("")

ObPlan=ipw.VBox([ObPlan_btn, OutputMessage5])
ObPlan.layout.align_items = 'center'


#DASHBOARD_3
DashBoard3=ipw.VBox([VOID,ObservationPlan_text, TimeStepUnit, ObPlan])
DashBoard3.layout.display = 'None'
display(DashBoard3)
